Ref: https://tf.wiki/zh_hant/deployment/export.html

https://www.tensorflow.org/api_docs/python/tf/saved_model

In [1]:
!git clone https://github.com/snowkylin/tensorflow-handbook
!mv ./tensorflow-handbook/source/_static/code/zh .

Cloning into 'tensorflow-handbook'...
remote: Enumerating objects: 14489, done.
remote: Counting objects: 100% (2731/2731), done.
remote: Compressing objects: 100% (1050/1050), done.
remote: Total 14489 (delta 2092), reused 2221 (delta 1672), pack-reused 11758
Receiving objects: 100% (14489/14489), 96.52 MiB | 31.00 MiB/s, done.
Resolving deltas: 100% (10192/10192), done.
Checking out files: 100% (2328/2328), done.


In [4]:
import tensorflow as tf
from zh.model.utils import MNISTLoader

num_epochs = 1
batch_size = 50
learning_rate = 0.001

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Softmax()
])

data_loader = MNISTLoader()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=[tf.keras.metrics.sparse_categorical_accuracy]
)

model.fit(data_loader.train_data, data_loader.train_label, epochs=num_epochs, batch_size=batch_size)
tf.saved_model.save(model, "saved/1")

1200/1200 [==============================] - 4s 3ms/step - loss: 0.2995 - sparse_categorical_accuracy: 0.9163
INFO:tensorflow:Assets written to: saved/1/assets


In [6]:
import tensorflow as tf
from zh.model.utils import MNISTLoader

batch_size = 50

model = tf.saved_model.load("saved/1")
data_loader = MNISTLoader()
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
print('num_batches:' + str(num_batches))

for batch_index in range(num_batches):
    # print('batch_index:' + str(batch_index))
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)

print("test accuracy: %f" % sparse_categorical_accuracy.result())

num_batches:200
test accuracy: 0.951400
